In [1]:
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [7]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000/ -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 7518210929 (7.00 GB)
DFS Remaining: 7251161088 (6.75 GB)
DFS Used: 267049841 (254.68 MB)
DFS Used%: 3.55%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 81
	Missing blocks (with replication factor 1): 81
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.22.0.5:9866 (project-4-team_bds_p4-dn-2.project-4-team_bds_p4_default)
Hostname: 6ac7f31c9295
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 267049841 (254.68 MB)
Non DFS Used: 18286064783 (17.03 GB)
DFS Remaining: 7251161088 (6.75 GB)
DFS Used%: 1.03%
DFS

In [3]:
import time
time.sleep(30)

In [8]:
#q9
r_total = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
total_size = r_total.json()['FileStatus']['length']
block_size = r_total.json()['FileStatus']['blockSize']
blocks_count = {'lost': 0}
i = 0
while(i*block_size < total_size):
    url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(block_size*i)
    r0 = requests.get(url)
    info = r0.json()
    if r0.status_code == 403:
        blocks_count['lost'] += 1
    else:
        host = info['Location'].split("/")[2][:-5]
        if host in blocks_count:
            blocks_count[host] += 1
        else:
            blocks_count[host] = 1
    i+=1
blocks_count

{'lost': 81, '6ac7f31c9295': 86}

In [9]:
#q10
# r_total = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
# total_size = r_total.json()['FileStatus']['length']
# block_size = r_total.json()['FileStatus']['blockSize']

hdfs = pa.fs.HadoopFileSystem("boss", 9000)
blocks_count = {'lost': 0}
i = 0
total_occrs = 0
with hdfs.open_input_file("/single.csv") as f:
    while(i*block_size < total_size):
        url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(block_size*i)
        r0 = requests.get(url)
        info = r0.json()
        if r0.status_code == 403:
            blocks_count['lost'] += 1
        else:
            host = info['Location'].split("/")[2][:-5]
            if host in blocks_count:
                blocks_count[host] += 1
            else:
                blocks_count[host] = 1
            block = f.read_at(1048576,1048576*i)
            #reader = io.TextIOWrapper(io.BufferedReader(block))
            reader = block.decode()
            total_occrs += reader.count("Single Family")
        i+=1
total_occrs

2023-10-28 00:24:25,477 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


228596